In [ ]:
from bertopic import BERTopic
import pandas as pd
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer

DATA_FLOW = 'local'
DATA_SAMPLE = 0.05

def preprocess(text):
    text = text.replace('\n', ' ')
    text = text.replace(u'\xa0', u' ')
    text = text.replace('@', '')
    new_text = []
    for t in text.split(" "):
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Read news data
if DATA_FLOW == 'local:'
    df = pd.read_csv("./../data/raw/news_tweets.csv")
elif DATA_FLOW == 'gcp':
    from google.cloud import storage
    from io import BytesIO
    client = storage.Client()
    bucket_name = "covid-news-response"
    file_name = "news_tweets.csv"
    bucket = client.get_bucket(bucket_name)
    blob = bucket.get_blob(file_name)
    content = blob.download_as_string()
    df = pd.read_csv(BytesIO(content))

# Clear news data
df['clean_text'] = df.text.apply(lambda x: preprocess(x))
docs = df.sample(frac=DATA_SAMPLE, random_state=42).clean_text.values
quantity = len(docs)